<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/DL_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

fatal: destination path 'Qualcomm-DL-Hackathon' already exists and is not an empty directory.


In [ ]:
import os
import zipfile

In [20]:
import zipfile
import os
import shutil

def extract_files_only(zip_file_paths, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for zip_file_path in zip_file_paths:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Check if the member is a file (not a directory)
                if not member.endswith('/'):
                    # Extract the file to a temporary location
                    source = zip_ref.open(member)
                    target_path = os.path.join(destination_folder, os.path.basename(member))
                    with open(target_path, "wb") as target:
                        shutil.copyfileobj(source, target)

# Example usage
zip_file_paths = ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip']
destination_folder = '/content/Qualcomm-DL-Hackathon/Images'

extract_files_only(zip_file_paths, destination_folder)
print(f"Files from {zip_file_paths} have been extracted to {destination_folder}.")

Files from ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip'] have been extracted to /content/Qualcomm-DL-Hackathon/Images.


In [21]:
def count_files_in_folder(folder_path):
    with os.scandir(folder_path) as entries:
        return sum(1 for entry in entries if entry.is_file())

# Example usage
folder_path = '/content/Qualcomm-DL-Hackathon/Images'
file_count = count_files_in_folder(folder_path)
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 2352
